# URL Params for the Image-Specific Endpoint

Make sure to check out the documentation [here]().

If you've used the Bing Search APIs before, you may be familiar with the `responseFilter=Images` URL parameter. This is useful if you **only** have access to the websearch endpoint. 

However, if you have access to the image endpoint, you can use a much richer set of filters to get the pictures you're looking for & get much richer metadata.

# Index
- [Setup](#Setup)
- [Exploring params of the Image-Specific search endpoint](#Exploring-params-of-the-Image-Specific-search-endpoint)
- [Practice](#Practice)

## Setup

In [1]:
import os
import requests
from IPython.display import Image
from pprint import pprint

In [2]:
API_KEY = os.environ.get('BING_KEY', default='ENTER YOUR API KEY HERE IF YOU DIDNT SET BING_KEY')

Double check that the cell below prints you Bing API key:

In [36]:
print(API_KEY)

Some setup:

In [4]:
ENDPOINT = ' https://api.cognitive.microsoft.com/bing/v7.0/images/search?'
HEADERS = {
    'Ocp-Apim-Subscription-Key' : API_KEY
}
STARTING_PARAMS = {
    "q" : None,
    "mkt" : "en-US",
    "offset" : "0",
  }

In [5]:
# Common accross all notebooks - 
def call_api(query, session=None, custom_params=None):
    sesh = session
    if not session:
        sesh = requests.Session()
        sesh.params = PARAMS
        sesh.params.update({'q' : query})
        sesh.headers = HEADERS
    if custom_params:
        sesh.params.update(custom_params)
    return sesh.get(ENDPOINT)

def handle_response(resp):
    assert resp.status_code == 200
    return resp.json()

In [6]:
# Common accross image-centric notebooks - 

def extract_with_args(response_json, *args):
    return [[j[i] for i in args] for j in response_json['value']]

def extract_with_single_arg(response_json, arg):
    return [i[arg] for i in response_json['value']]

def extract_with_deep_arg(response_json, arg_parent, arg):
    return [i[arg_parent][arg] for i in response_json['value']]

def extract_img_urls(response_json):
    return [i['contentUrl'] for i in response_json['value']]

def extract_img_entries(response_json):
    return [i for i in response_json['value']]

def extract_img_dimens(response_json):
    return [(i['height'], i['width']) for i in response_json['value']]

def display(img):
    return Image(url=img)

## Exploring params of the Image-Specific search endpoint

#### Ex 1: Square drawing of a red car that's free to share commercially

First, let's get a copy of our default URL param dictionary, & just do a normal search for images of red cars.

In [7]:
q = 'red car'
PARAMS = STARTING_PARAMS.copy()

Simple enough. The functions above will ask for pictures of `'red car'`s and give us back some URLs. Let's create a few functions that help us tie the three helper functions above together:

In [8]:
def get_json_from_bing(q, session=None, custom_params=None):
    resp = call_api(q, session=session, custom_params=custom_params)
    json = handle_response(resp)
    return json

def get_metadata(q, session=None, custom_params=None, metadata_types=()):
    json = get_json_from_bing(q, session=session, custom_params=custom_params)
    if len(metadata_types) == 1:
        return extract_with_single_arg(json, metadata_types[0])
    return extract_with_args(json, *metadata_types)
    
def get_images(q, session=None, custom_params=None):
    json = get_json_from_bing(q, session=session, custom_params=custom_params)
    return extract_img_urls(json)

Now it's easy for us to get images from Bing using a simple search term:

In [9]:
red_car_json = extract_img_entries(get_json_from_bing(q))
images = get_images(q)

And voila!

In [10]:
display(images[0])

And here's the full response json entry  that we're extracting this image url from:

In [11]:
pprint(red_car_json[0])

{'accentColor': 'BF0C0C',
 'contentSize': '67929 B',
 'contentUrl': 'https://img.huffingtonpost.com/asset/598cc71515000084208b6139.jpg?ops=scalefit_820_noupscale',
 'datePublished': '2019-03-24T20:20:00.0000000Z',
 'encodingFormat': 'jpeg',
 'height': 544,
 'hostPageDisplayUrl': 'https://www.huffingtonpost.com/entry/negative-body-image-and-the-red-car-syndrome_us...',
 'hostPageUrl': 'https://www.huffingtonpost.com/entry/negative-body-image-and-the-red-car-syndrome_us_598cc58be4b0caa1687a5eb6',
 'imageId': '2E1DF39CD461570C0396619E0162BE5499B0339E',
 'imageInsightsToken': 'ccid_QMiV/e2C*mid_2E1DF39CD461570C0396619E0162BE5499B0339E*simid_608055716015837139*thid_OIP.QMiV!_e2C9Nw6PfmN8n2PfgHaE6',
 'insightsMetadata': {'availableSizesCount': 7, 'pagesIncludingCount': 9},
 'isFamilyFriendly': True,
 'name': 'Negative Body Image and ‘The Red Car Syndrome’ | HuffPost',
 'thumbnail': {'height': 314, 'width': 474},
 'thumbnailUrl': 'https://tse3.mm.bing.net/th?id=OIP.QMiV_e2C9Nw6PfmN8n2PfgHaE6&

If you `display()` all the images in `images`, you'll notice that they are mostly real photos. Let's say we're looking for **drawings** of red cars.

Since we're using the image-specific API endpoint, we can pass in params like `'imageType'`

In [12]:
new_param = {'imageType' : 'Clipart'}
images = get_images(q, custom_params=new_param)

In [13]:
display(images[0])

Ok cool. But now let's pretend I'm looking for images to use in my website & need one that's a perfect square. 

In [14]:
new_new_param = {'aspect' : 'square'}
images = get_images(q, custom_params=new_new_param)

In [15]:
display(images[0])

In [16]:
display(images[-1])

Oh no! That last one looks super-proprietary. I'd like also to only find images that are ok for me to use commercially.

In [18]:
newer_params = {
    'license' : 'ShareCommercially',
    'color' : 'Red'
}
images = get_images(q, custom_params=newer_params)

In [19]:
print(images[0])
display(images[0])

https://upload.wikimedia.org/wikipedia/commons/2/2b/Black_Man_Driving_Car_GIF_Animation_Loop.gif


Note that, while useful for free-content aggregation, certain license params can reduce the number of relevant results.

For example, we're now looking for square & non-photographic images of `'red car'`s which their owners have manually tagged as "free to use commercially." Let's see what happens when I try to add `{'size' : 'large'}`:

In [20]:
newest_params = {
    'size' : 'large'
}
images = get_images(q, custom_params=newest_params)

In [21]:
display(images[0])

## Practice

#### Pictures of People

Let's find large faceshots of people smiling.

In [22]:
q = 'smiling'
PARAMS = STARTING_PARAMS.copy()

In [23]:
new_params = {
    'size' : 'Large',
    'imageType' : 'Photo',
    'imageContent' : 'Face' 
}

In [24]:
images = get_images(q, custom_params=new_params)

In [25]:
print(images[0])
display(images[0])

http://primedental.files.wordpress.com/2012/01/man-smiling-in-black-shirt.jpg


#### The Empire State Building

Let's find a tall photo of the empire state building at night

In [26]:
q = 'empire state building at midnight'
PARAMS = STARTING_PARAMS.copy()

In [27]:
new_params = {
    'imageType' : 'Photo',
    'aspect' : 'tall'
}

In [28]:
images = get_images(q, custom_params=new_params)

In [29]:
print(images[1])
display(images[1])

http://media-cdn.tripadvisor.com/media/photo-s/05/40/fd/79/empire-state-building.jpg


Simple enough. 

#### Blue Pandas

Now I want blue a drawing of an adorable panda bear.

In [30]:
PARAMS = STARTING_PARAMS.copy()
q = 'panda bear drawing'
even_newer_params = {
    'color' : 'Blue',
    'size' : 'Wallpaper'
}

In [31]:
images = get_images(q, custom_params=even_newer_params)

In [32]:
display(images[0])

Blue? Check. Panda Bear? Check. Adorable? I mean at least while I'm running it: Check.

Now let's get some metadata about the image:

In [33]:
params_i_want = ['height', 'width', 'contentSize', 'accentColor']

In [34]:
dataz = get_metadata(q, custom_params=even_newer_params, metadata_types=params_i_want)

In [35]:
print('The image above is {}px tall, {}px wide, its size is {}, and its dominant color is #{}.'.format(*dataz[0]))

The image above is 1080px tall, 1920px wide, its size is 99479 B, and its dominant color is #638E3D.


now let's get a thumbnail:

In [37]:
thumbs = ['thumbnailUrl']

In [38]:
thumb_urls = get_metadata(q, custom_params=even_newer_params, metadata_types=thumbs)
display(thumb_urls[0])

Move on to the Image insights Notebook